In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

from hyperas.distributions import uniform

import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import np_utils

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

Using TensorFlow backend.


In [2]:
def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """
    
    def create_dataset(dataset, look_back=1):

        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):

            #The target is always the next value. And the lookback are the previous prices
            a = dataset[i:(i+look_back), :]
            dataX.append(a)
            dataY.append(dataset[i + look_back, :][0])

        return np.array(dataX), np.array(dataY)

    df = pd.read_csv('tweet_n_btc_day.csv')
    #Drop np.nan
    df.dropna(how='any', inplace=True)

    #Make it array
    df = df[['Weighted_Price']]
    
    dataset = df.values
    dataset = dataset.astype('float32')
    
    # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    # split into train and test sets
    train_size = int(len(dataset) * 0.60)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    
    # reshape into X=t and Y=t+1
    look_back = 5
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))

    return trainX, trainY, testX, testY

In [3]:
def create_model(trainX, trainY, testX, testY):
    
    callbacks = [

            keras.callbacks.EarlyStopping(monitor='loss',patience=3),
            keras.callbacks.ModelCheckpoint(filepath='forecasting_price_model.h5', monitor='val_loss',save_best_only=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=1),
            keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
        ]

    # Define Model
    model = Sequential()
    model.add(LSTM( {{choice([4, 16, 32, 64, 128])}}, input_shape=(look_back, trainX.shape[2]),
                   dropout = {{uniform(0, 1)}}, recurrent_dropout={{uniform(0, 1)}},return_sequences=True
                  ))
    
    model.add(LSTM({{choice([4, 16, 32, 64, 128])}}, dropout = {{uniform(0, 1)}}, recurrent_dropout={{uniform(0, 1)}}))
    
    if conditional({{choice(['two', 'three'])}}) == 'two':
        model.add(Dense(512))
    
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer={{choice(['adam'])}} )
    model.fit(trainX, trainY, epochs=250, batch_size={{choice([1, 4, 16, 32, 64, 128])}},callbacks=callbacks,
                                                                                                  validation_split=0.25,)
    
    model.load_weights("forecasting_price_model.h5")
    
    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)

    # calculate root mean squared error
    trainScore = math.sqrt(mean_squared_error(trainY[:], trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[:], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    

    
    return {'loss': trainScore, 'status': STATUS_OK, 'model': model}

In [4]:
best_run, best_model = optim.minimize(model=create_model,
                                              data=data,
                                              algo=tpe.suggest,
                                              max_evals=250,
                                              trials=Trials(),notebook_name='5. Optimize Bitcoin price prediction with LSTM')

X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)
print(best_model.evaluate(X_train, Y_test))

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import math
except:
    pass

try:
    from sklearn.preprocessing import normalize, MinMaxScaler
except:
    pass

try:
    from sklearn.svm import SVR
except:
    pass

try:
    from sklearn.metrics import mean_squared_error
except:
    pass

try:
    from sklearn.ensemble import RandomForestRegressor
except:
    pass

try:
    from hyperas.distributions import uniform
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import LSTM, Dense
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activat

Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 2s 10ms/step - loss: 0.0011 - val_loss: 0.0026
Epoch 2/250
159/159 [==============================] - 0s 985us/step - loss: 8.7839e-04 - val_loss: 0.0045
Epoch 3/250
159/159 [==============================] - 0s 953us/step - loss: 7.0418e-04 - val_loss: 0.0029
Epoch 4/250
159/159 [==============================] - 0s 979us/step - loss: 7.0204e-04 - val_loss: 0.0029
Epoch 00004: early stopping
Train Score: 0.04 RMSE
Test Score: 0.38 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 4s 27ms/step - loss: 9.6317e-04 - val_loss: 0.0064
Epoch 2/250
159/159 [==============================] - 3s 16ms/step - loss: 9.0606e-04 - val_loss: 0.0050
Epoch 3/250
159/159 [==============================] - 3s 16ms/step - loss: 9.1075e-04 - val_loss: 0.0037
Epoch 4/250
159/159 [==============================] - 3s 16ms/step - loss: 9.4012e-04 - val_los

Train Score: 0.03 RMSE
Test Score: 0.36 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 3s 20ms/step - loss: 0.0014 - val_loss: 0.0040
Epoch 2/250
159/159 [==============================] - 0s 314us/step - loss: 8.0705e-04 - val_loss: 0.0019
Epoch 3/250
159/159 [==============================] - 0s 289us/step - loss: 9.1065e-04 - val_loss: 0.0027
Epoch 4/250
159/159 [==============================] - 0s 308us/step - loss: 7.6968e-04 - val_loss: 0.0040
Epoch 5/250
159/159 [==============================] - 0s 301us/step - loss: 7.8472e-04 - val_loss: 0.0040
Epoch 00005: early stopping
Train Score: 0.03 RMSE
Test Score: 0.35 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 3s 22ms/step - loss: 0.0012 - val_loss: 0.0031
Epoch 2/250
159/159 [==============================] - 0s 1ms/step - loss: 9.2193e-04 - val_loss: 0.0041
Epoch 3/250
159/159 [==============================] - 

Epoch 11/250
159/159 [==============================] - 0s 1ms/step - loss: 8.6285e-05 - val_loss: 1.6547e-04
Epoch 12/250
159/159 [==============================] - 0s 1ms/step - loss: 7.6016e-05 - val_loss: 1.5853e-04
Epoch 13/250
159/159 [==============================] - 0s 1ms/step - loss: 5.7930e-05 - val_loss: 1.5765e-04
Epoch 00013: early stopping
Train Score: 0.01 RMSE
Test Score: 0.09 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 5s 29ms/step - loss: 0.0012 - val_loss: 0.0071
Epoch 2/250
159/159 [==============================] - 0s 1ms/step - loss: 8.5665e-04 - val_loss: 0.0041
Epoch 3/250
159/159 [==============================] - 0s 1ms/step - loss: 7.5230e-04 - val_loss: 0.0013
Epoch 4/250
159/159 [==============================] - 0s 985us/step - loss: 5.1216e-04 - val_loss: 3.7936e-04
Epoch 5/250
159/159 [==============================] - 0s 966us/step - loss: 5.0175e-04 - val_loss: 4.7863e-04
Epoch 6/250
159/15

Epoch 5/250
159/159 [==============================] - 0s 1ms/step - loss: 4.2354e-04 - val_loss: 7.2088e-04
Epoch 6/250
159/159 [==============================] - 0s 1ms/step - loss: 4.4669e-04 - val_loss: 3.6120e-04
Epoch 7/250
159/159 [==============================] - 0s 1ms/step - loss: 4.2171e-04 - val_loss: 4.2641e-04
Train Score: 0.02 RMSE
Test Score: 0.16 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 6s 40ms/step - loss: 0.0013 - val_loss: 0.0019
Epoch 2/250
159/159 [==============================] - 0s 301us/step - loss: 0.0011 - val_loss: 0.0040
Epoch 3/250
159/159 [==============================] - 0s 295us/step - loss: 8.0842e-04 - val_loss: 0.0057
Epoch 4/250
159/159 [==============================] - 0s 315us/step - loss: 9.6200e-04 - val_loss: 0.0054
Epoch 00004: early stopping
Train Score: 0.04 RMSE
Test Score: 0.37 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [=========================

...

Epoch 6/250
159/159 [==============================] - 1s 5ms/step - loss: 2.4441e-04 - val_loss: 3.4690e-04
Epoch 7/250
159/159 [==============================] - 1s 5ms/step - loss: 4.1356e-04 - val_loss: 2.0325e-04
Epoch 8/250
159/159 [==============================] - 1s 6ms/step - loss: 3.2550e-04 - val_loss: 2.8978e-04
Epoch 9/250
159/159 [==============================] - 1s 5ms/step - loss: 3.7444e-04 - val_loss: 5.7147e-04
Train Score: 0.01 RMSE
Test Score: 0.08 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 44s 280ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 2/250
159/159 [==============================] - 0s 414us/step - loss: 0.0011 - val_loss: 0.0045
Epoch 3/250
159/159 [==============================] - 0s 408us/step - loss: 8.2832e-04 - val_loss: 0.0041
Epoch 4/250
159/159 [==============================] - 0s 408us/step - loss: 7.6723e-04 - val_loss: 0.0038
Epoch 00004: early stopping
Train Score: 0.04 RMSE
Te

159/159 [==============================] - 0s 1ms/step - loss: 3.2870e-04 - val_loss: 4.3737e-04
Epoch 00006: early stopping
Train Score: 0.01 RMSE
Test Score: 0.06 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 46s 292ms/step - loss: 0.0014 - val_loss: 0.0038
Epoch 2/250
159/159 [==============================] - 0s 420us/step - loss: 8.2559e-04 - val_loss: 0.0016
Epoch 3/250
159/159 [==============================] - 0s 395us/step - loss: 9.2451e-04 - val_loss: 0.0023
Epoch 4/250
159/159 [==============================] - 0s 414us/step - loss: 6.4061e-04 - val_loss: 0.0036
Epoch 5/250
159/159 [==============================] - 0s 439us/step - loss: 6.7473e-04 - val_loss: 0.0037
Epoch 00005: early stopping
Train Score: 0.03 RMSE
Test Score: 0.34 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 46s 291ms/step - loss: 8.9343e-04 - val_loss: 0.0021
Epoch 2/250
159/159 [======

159/159 [==============================] - 1s 5ms/step - loss: 5.9818e-04 - val_loss: 0.0018
Epoch 3/250
159/159 [==============================] - 1s 5ms/step - loss: 4.6882e-04 - val_loss: 0.0018
Epoch 4/250
159/159 [==============================] - 1s 5ms/step - loss: 5.9080e-04 - val_loss: 0.0020
Epoch 5/250
159/159 [==============================] - 1s 6ms/step - loss: 4.8821e-04 - val_loss: 0.0011
Epoch 6/250
159/159 [==============================] - 1s 6ms/step - loss: 4.2009e-04 - val_loss: 0.0012
Epoch 7/250
159/159 [==============================] - 1s 6ms/step - loss: 4.7106e-04 - val_loss: 0.0014
Epoch 8/250
159/159 [==============================] - 1s 5ms/step - loss: 5.7983e-04 - val_loss: 0.0012
Epoch 00008: early stopping
Train Score: 0.02 RMSE
Test Score: 0.22 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/159 [==============================] - 47s 299ms/step - loss: 0.0010 - val_loss: 0.0025
Epoch 2/250
159/159 [==============================] - 

Epoch 2/250
159/159 [==============================] - 0s 1ms/step - loss: 7.2183e-04 - val_loss: 0.0024
Epoch 3/250
159/159 [==============================] - 0s 1ms/step - loss: 4.6074e-04 - val_loss: 0.0016
Epoch 4/250
159/159 [==============================] - 0s 1ms/step - loss: 3.4153e-04 - val_loss: 5.1847e-04
Epoch 5/250
159/159 [==============================] - 0s 1ms/step - loss: 2.9107e-04 - val_loss: 3.1088e-04
Epoch 6/250
159/159 [==============================] - 0s 1ms/step - loss: 3.4873e-04 - val_loss: 2.2296e-04
Epoch 7/250
159/159 [==============================] - 0s 2ms/step - loss: 1.9603e-04 - val_loss: 2.2613e-04
Epoch 8/250
159/159 [==============================] - 0s 2ms/step - loss: 1.8192e-04 - val_loss: 2.5241e-04
Epoch 9/250
159/159 [==============================] - 0s 1ms/step - loss: 2.0413e-04 - val_loss: 2.5603e-04
Epoch 00009: early stopping
Train Score: 0.01 RMSE
Test Score: 0.10 RMSE
Train on 159 samples, validate on 54 samples
Epoch 1/250
159/15

ValueError: Input arrays should have the same number of samples as target arrays. Found 213 input samples and 140 target samples.